In [4]:
import mdtraj as md
import numpy as np
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster, cophenet
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics import silhouette_samples, silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.spatial.distance import pdist
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar la trayectoria
traj = md.load('traj.pdb')  # Asegúrate de que el archivo esté en formato PDB o compatible con MDTraj

# Calcular la matriz de distancias usando RMSD entre todas las conformaciones
num_frames = traj.n_frames
distance_matrix = np.zeros((num_frames, num_frames))

for i in range(num_frames):
    rmsd_values = md.rmsd(traj, traj, frame=i)
    distance_matrix[i, :] = rmsd_values
    distance_matrix[:, i] = rmsd_values  # Simetría en la matriz de distancias
    
# Calcular las distancias promedio de cada conformación
avg_distances = np.mean(distance_matrix, axis=1)
    
# Calcular los percentiles
lower_threshold = np.percentile(avg_distances, 1)
upper_threshold = np.percentile(avg_distances, 99)
    
# Filtrar las conformaciones que están fuera de los percentiles
non_outliers = (avg_distances >= lower_threshold) & (avg_distances <= upper_threshold)
    
# Aplicar el filtro a la matriz de distancias
filtered_distance_matrix = distance_matrix[non_outliers][:, non_outliers]

####################################################################################
#####COMENTAR ESTA LINEA SI QUIERES EJECUTAR TODO COMO LO TENIAS SIN EL OUTLIER#####
####################################################################################
distance_matrix = filtered_distance_matrix

# Visualización de la matriz de distancias
sns.heatmap(distance_matrix, cmap='viridis')
plt.title("Matriz de Distancias (RMSD)")
plt.show()

# Clustering con K-means
num_clusters = 5  # Ajusta según el análisis de la matriz de distancias o dendrogramas previos
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
labels_kmeans = kmeans.fit_predict(distance_matrix)

# Imprimir etiquetas de cluster K-means
print("Etiquetas de Cluster - K-means:", labels_kmeans)

# Visualización de la matriz de distancias
sns.heatmap(distance_matrix, cmap='viridis')
plt.title("Matriz de Distancias (RMSD)")
plt.show()

# Clustering jerárquico
threshold = 0.3  # Ajustar según los resultados observados
linkage_matrix = linkage(squareform(distance_matrix), method='average')
labels_hierarchical = fcluster(linkage_matrix, threshold, criterion='distance')

# Imprimir etiquetas de cluster
print("Etiquetas de Cluster - Agrupamiento Jerárquico:", labels_hierarchical)

# DBSCAN con eps ajustado
eps_value = 0.05  # Ajustar eps según los resultados
dbscan_model = DBSCAN(eps=eps_value, min_samples=2, metric='precomputed')
labels_dbscan = dbscan_model.fit_predict(distance_matrix)

# Imprimir etiquetas de cluster DBSCAN
print("Etiquetas de Cluster - DBSCAN:", labels_dbscan)

# Clustering espectral (Spectral Clustering)
spectral = SpectralClustering(n_clusters=num_clusters, affinity='precomputed', random_state=0)
labels_spectral = spectral.fit_predict(distance_matrix)

# Imprimir etiquetas de cluster Spectral Clustering
print("Etiquetas de Cluster - Clustering Espectral:", labels_spectral)

# Visualización de los resultados de clustering
plt.figure(figsize=(15, 10))

# KMeans clustering
plt.subplot(2, 2, 1)
sns.scatterplot(x=np.arange(num_frames), y=labels_kmeans, palette="viridis", marker="o")
plt.title("Etiquetas K-means")
plt.xlabel("Conformación")
plt.ylabel("Cluster")

# Spectral clustering
plt.subplot(2, 2, 2)
sns.scatterplot(x=np.arange(num_frames), y=labels_spectral, palette="viridis", marker="o")
plt.title("Etiquetas Clustering Espectral")
plt.xlabel("Conformación")
plt.ylabel("Cluster")

# DBSCAN clustering
plt.subplot(2, 2, 3)
sns.scatterplot(x=np.arange(num_frames), y=labels_dbscan, palette="viridis", marker="o")
plt.title("Etiquetas Clustering DBSCAN")
plt.xlabel("Conformación")
plt.ylabel("Cluster")

# Hierarchical clustering
plt.subplot(2, 2, 4)
sns.scatterplot(x=np.arange(num_frames), y=labels_hierarchical, palette="viridis", marker="o")
plt.title("Etiquetas Clustering Hierarchical")
plt.xlabel("Conformación")
plt.ylabel("Cluster")

plt.tight_layout()
plt.show()

# Encontrar la conformación representativa (medoid) para cada cluster en K-means
unique_clusters_kmeans = np.unique(labels_kmeans)
medoids_kmeans = {}
for cluster in unique_clusters_kmeans:
    indices = np.where(labels_kmeans == cluster)[0]
    sub_matrix = distance_matrix[np.ix_(indices, indices)]
    medoid_index = indices[np.argmin(sub_matrix.sum(axis=0))]
    medoids_kmeans[cluster] = medoid_index

print("Medoids para agrupamiento K-means:", medoids_kmeans)

# Encontrar la conformación representativa (medoid) para cada cluster en Spectral Clustering
unique_clusters_spectral = np.unique(labels_spectral)
medoids_spectral = {}
for cluster in unique_clusters_spectral:
    indices = np.where(labels_spectral == cluster)[0]
    sub_matrix = distance_matrix[np.ix_(indices, indices)]
    medoid_index = indices[np.argmin(sub_matrix.sum(axis=0))]
    medoids_spectral[cluster] = medoid_index

print("Medoids para agrupamiento Clustering Espectral:", medoids_spectral)

# Encontrar la conformación representativa (medoid) para cada cluster en agrupamiento jerárquico
unique_clusters = np.unique(labels_hierarchical)
medoids_hierarchical = {}
for cluster in unique_clusters:
    indices = np.where(labels_hierarchical == cluster)[0]
    sub_matrix = distance_matrix[np.ix_(indices, indices)]
    medoid_index = indices[np.argmin(sub_matrix.sum(axis=0))]  # Índice del medoid
    medoids_hierarchical[cluster] = medoid_index

print("Medoids para agrupamiento jerárquico:", medoids_hierarchical)

# Encontrar la conformación representativa (medoid) para cada cluster en DBSCAN
unique_clusters_dbscan = np.unique(labels_dbscan)
medoids_dbscan = {}
for cluster in unique_clusters_dbscan:
    if cluster != -1:  # Ignorar puntos de ruido
        indices = np.where(labels_dbscan == cluster)[0]
        sub_matrix = distance_matrix[np.ix_(indices, indices)]
        medoid_index = indices[np.argmin(sub_matrix.sum(axis=0))]
        medoids_dbscan[cluster] = medoid_index

print("Medoids para agrupamiento DBSCAN:", medoids_dbscan)

"""
Índice de Silueta:

Cómo funciona: 
Evalúa cada punto comparando la distancia media a los puntos dentro del mismo clúster con la distancia media a los puntos en el clúster más cercano.

En qué se enfoca: 
En la separación de los clústeres y la cohesión interna.

Valores interpretativos: 
Rango de -1 a 1. Valores cercanos a 1 indican buen agrupamiento, 0 indica puntos en el borde de clústeres, y valores negativos indican posible mala asignación.

Índice de Davies-Bouldin:

Cómo funciona: 
Mide la relación entre la dispersión dentro de los clústeres y la distancia entre los clústeres.

En qué se enfoca: 
En la compacidad de los clústeres y la separación entre ellos.

Valores interpretativos: 
Valores positivos donde un valor más bajo indica clústeres más compactos y bien separados.

Índice de Calinski-Harabasz:

Cómo funciona: 
Calcula la razón entre la suma de las dispersiones entre los clústeres y la suma de las dispersiones dentro de los clústeres.

En qué se enfoca: 
En la dispersión entre y dentro de los clústeres.

Valores interpretativos: 
Valores positivos donde un valor más alto indica mejor calidad del clustering.
    """

# Calcular y graficar el silhouette para KMeans
silhouette_vals_kmeans = silhouette_samples(distance_matrix, labels_kmeans, metric='precomputed')
silhouette_avg_kmeans = silhouette_score(distance_matrix, labels_kmeans, metric='precomputed')
plt.figure(figsize=(10, 6))
y_lower = 10
for i in np.unique(labels_kmeans):
    ith_cluster_silhouette_values = silhouette_vals_kmeans[labels_kmeans == i]
    ith_cluster_silhouette_values.sort()
    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i
    plt.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, alpha=0.7)
    plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10
plt.title('Silhouette plot for KMeans clustering')
plt.xlabel('Silhouette coefficient')
plt.ylabel('Cluster')
plt.axvline(x=silhouette_avg_kmeans, color="red", linestyle="--")
plt.show()

# Calcular y graficar el silhouette para Spectral
silhouette_vals_spectral = silhouette_samples(distance_matrix, labels_spectral, metric='precomputed')
silhouette_avg_spectral = silhouette_score(distance_matrix, labels_spectral, metric='precomputed')
plt.figure(figsize=(10, 6))
y_lower = 10
for i in np.unique(labels_spectral):
    ith_cluster_silhouette_values = silhouette_vals_spectral[labels_spectral == i]
    ith_cluster_silhouette_values.sort()
    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i
    plt.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, alpha=0.7)
    plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10
plt.title('Silhouette plot for Spectral clustering')
plt.xlabel('Silhouette coefficient')
plt.ylabel('Cluster')
plt.axvline(x=silhouette_avg_spectral, color="red", linestyle="--")
plt.show()

# Calcular y graficar el silhouette para DBSCAN
silhouette_vals_dbscan = silhouette_samples(distance_matrix, labels_dbscan, metric='precomputed')
silhouette_avg_dbscan = silhouette_score(distance_matrix, labels_dbscan, metric='precomputed')
plt.figure(figsize=(10, 6))
y_lower = 10
for i in np.unique(labels_dbscan):
    ith_cluster_silhouette_values = silhouette_vals_dbscan[labels_dbscan == i]
    ith_cluster_silhouette_values.sort()
    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i
    plt.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, alpha=0.7)
    plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10
plt.title('Silhouette plot for DBSCAN clustering')
plt.xlabel('Silhouette coefficient')
plt.ylabel('Cluster')
plt.axvline(x=silhouette_avg_dbscan, color="red", linestyle="--")
plt.show()

# Calcular y graficar el silhouette para Hierarchical
silhouette_vals_hierarchical = silhouette_samples(distance_matrix, labels_hierarchical, metric='precomputed')
silhouette_avg_hierarchical = silhouette_score(distance_matrix, labels_hierarchical, metric='precomputed')
plt.figure(figsize=(10, 6))
y_lower = 10
for i in np.unique(labels_hierarchical):
    ith_cluster_silhouette_values = silhouette_vals_hierarchical[labels_hierarchical == i]
    ith_cluster_silhouette_values.sort()
    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i
    plt.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, alpha=0.7)
    plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10
plt.title('Silhouette plot for Hierarchical clustering')
plt.xlabel('Silhouette coefficient')
plt.ylabel('Cluster')
plt.axvline(x=silhouette_avg_hierarchical, color="red", linestyle="--")
plt.show()

# Calcular el índice de Davies-Bouldin para cada método de clustering
db_index_kmeans = davies_bouldin_score(distance_matrix, labels_kmeans)
db_index_spectral = davies_bouldin_score(distance_matrix, labels_spectral)
db_index_dbscan = davies_bouldin_score(distance_matrix, labels_dbscan)
db_index_hierarchical = davies_bouldin_score(distance_matrix, labels_hierarchical)

print(f"Davies-Bouldin Index for KMeans: {db_index_kmeans}")
print(f"Davies-Bouldin Index for Spectral: {db_index_spectral}")
print(f"Davies-Bouldin Index for DBSCAN: {db_index_dbscan}")
print(f"Davies-Bouldin Index for Hierarchical: {db_index_hierarchical}")

# Calcular el índice de Calinski-Harabasz para cada algoritmo
calinski_kmeans = calinski_harabasz_score(distance_matrix, labels_kmeans)
calinski_dbscan = calinski_harabasz_score(distance_matrix, labels_dbscan)
calinski_spectral = calinski_harabasz_score(distance_matrix, labels_spectral)
calinski_hierarchical = calinski_harabasz_score(distance_matrix, labels_hierarchical)

print("Índice de Calinski-Harabasz para K-means:", calinski_kmeans)
print("Índice de Calinski-Harabasz para DBSCAN:", calinski_dbscan)
print("Índice de Calinski-Harabasz para Spectral Clustering:", calinski_spectral)
print("Índice de Calinski-Harabasz para Clustering Jerárquico:", calinski_hierarchical)


"""# Calcular el coeficiente de correlación de cophenético para clustering hierarchical
Z = linkage(distance_matrix, method='ward')
coph_corr, _ = cophenet(Z, pdist(distance_matrix))
print(f"Cophenetic Correlation Coefficient for Hierarchical Clustering: {coph_corr}")"""



ModuleNotFoundError: No module named 'mdtraj'